In [16]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import math
import pandas as pd
import json
from tqdm import tqdm
import ast
import nltk
from nltk.corpus import stopwords

In [4]:
#testing one off
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
input_text = "The quick brown fox jumps over the lazy dog."

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,  # or bfloat16 or int8/int4 if quantized
    device_map="auto"
)
model.eval()

inputs = tokenizer(input_text, return_tensors="pt").to(model.device)
input_ids = inputs["input_ids"]
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits
shift_logits = logits[:, :-1, :]
shift_labels = input_ids[:, 1:]
log_probs = torch.nn.functional.log_softmax(shift_logits, dim=-1)
token_log_probs = log_probs.gather(2, shift_labels.unsqueeze(-1)).squeeze(-1)
average_log_prob = token_log_probs.mean().item()
print(f"Input: {input_text}")
print(f"Average Log Probability: {average_log_prob:.4f}")
print(f"Per-token Log Probs: {[round(lp.item(), 4) for lp in token_log_probs[0]]}")

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Input: The quick brown fox jumps over the lazy dog.
Average Log Probability: -1.6318
Per-token Log Probs: [-10.1328, -8.3281, -0.0698, -0.0097, -0.002, -0.0336, -0.001, -0.027, -0.0308, -0.0165, -0.0222, -0.9102]


In [23]:
#data sources

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))


full_df = pd.read_csv("procon_longer.csv")
irdf = pd.read_csv("interpretive_repertoires.csv")

def clean_text(text):
    if not isinstance(text, str):
        return ""
    text = text.lower()
    tokens = text.split()
    tokens = [t for t in tokens if t not in stop_words]
    return " ".join(tokens)

full_df['raw_text'] = full_df['point'] + "\n" + full_df['explanation']
full_df["clean"] = full_df["raw_text"].apply(clean_text)

# full_df.head()
fdf = full_df[['title', 'stance', 'clean']].copy()

[nltk_data] Downloading package stopwords to
[nltk_data]     /ihome/xli/sek188/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [28]:
fdf.head()

,title,stance,clean
0,Filibuster,pro,pro 1: filibuster promotes compromise protects...
1,Filibuster,pro,pro 2: filibuster protects intended purpose se...
2,Filibuster,pro,pro 3: filibuster important safeguard politica...
3,Filibuster,con,con 1: filibuster promotes obstructionism part...
4,Filibuster,con,con 2: filibuster prevents meaningful debate s...


In [48]:
#stitching and liloing
combined_rows = []
for _, f_row in fdf.iterrows():
    for _, ir_row in irdf.iterrows():
        combined_row = {
            **f_row.to_dict(),      # all columns from fdf
            **ir_row.to_dict()      # all columns from irdf
        }
        combined_rows.append(combined_row)

cdf = pd.DataFrame(combined_rows)
cdf["attestation"] = cdf["likes"].combine_first(cdf["dislikes"])
cdf["input_text"] = cdf["attestation"] + "\n\n" + cdf["clean"]
cdf.drop(columns=['likes', 'dislikes', 'guidewords'])

funkyoutputblocker = None

In [49]:
#temp to test:
# cdf = cdf.head()

#model bit
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    device_map="auto"
)
model.eval()

# Compute logprobs
logprobs = []
for idx, row in tqdm(cdf.iterrows(), total=len(cdf)):
    inputs = tokenizer(row['input_text'], return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits

    input_ids = inputs["input_ids"]
    shift_logits = logits[:, :-1, :]
    shift_labels = input_ids[:, 1:]
    log_probs = torch.nn.functional.log_softmax(shift_logits, dim=-1)
    token_log_probs = log_probs.gather(2, shift_labels.unsqueeze(-1)).squeeze(-1)
    average_log_prob = token_log_probs.mean().item()
    logprobs.append(average_log_prob)

cdf["logprob"] = logprobs




100%|██████████| 9758/9758 [21:50:44<00:00,  8.06s/it]   


In [50]:
cdf.to_csv('procon_coh_long.csv')
cdf['logprob'].mean

<bound method Series.mean of 0      -3.229038
1      -3.173262
2      -3.139151
3      -3.127598
4      -3.019099
          ...   
9753   -3.156641
9754   -3.308724
9755   -3.100954
9756   -3.126193
9757   -3.292845
Name: logprob, Length: 9758, dtype: float64>